# Example of a Plasmid Restriction/Ligation Cloning
> Visit the full library documentation [here](https://bjornfjohansson.github.io/pydna/)

This example showcases a workflow of modelling molecular cloning with restriction enzymes, PCR, and ligases, to clone gene fragments into plasmids. This example constructs a synthetic plasmid by cloning the ase1 gene, which encodes a microtubule associated protein responsible for mitotic spindle assembly, into the pFA6a-kanMX6 cloning vector:

1. The ase1 gene fragment is first cloned from a portion of the _S. pombe_ genome through PCR:
2. The pFA6a-kanMX6 cloning vector is then cleaved with AscI and SalI. The ase1 gene fragment is also cleaved with SalI and AscI
3. The fragment is ligated with the linearized pFA6a-kanMX6 vector.

Source files can be found alongside this notebook, if you would like to follow along. Annotations are made alongside the code to describe key steps.

<a target="_blank" href="https://colab.research.google.com/github/BjornFJohansson/pydna/blob/dev_bjorn/docs/notebooks/Example_Restriction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# Importing all necessary classes and methods

from pydna.parsers import parse
from pydna.tm import tm_default
from pydna.amplify import pcr
from pydna.dseqrecord import Dseqrecord
from Bio.Restriction import SalI, AscI

In [ ]:
# Parsing the files
pFA6akanMX6_path = "./pFA6a-kanMX6.gb"
ase1_path = "./CU329670.gb"
vector = parse(pFA6akanMX6_path)[0]
pombe_chromosome_I = parse(ase1_path)[0]

# Printing the parsed files

print(vector.format("gb"))
print(pombe_chromosome_I.format("gb"))

LOCUS       pFA6a-kanMX6            3938 bp    ds-DNA  circular SYN 16-JUN-2022
DEFINITION  synthetic circular DNA.
ACCESSION   .
VERSION     .
KEYWORDS    pFA6a-kanMX6.
SOURCE      synthetic DNA construct
  ORGANISM  synthetic DNA construct
            .
REFERENCE   1  (bases 1 to 3938)
  AUTHORS   Bahler J, Wu JQ, Longtine MS, Shah NG, McKenzie A 3rd, Steever AB,
            Wach A, Philippsen P, Pringle JR
  TITLE     Heterologous modules for efficient and versatile PCR-based gene
            targeting in Schizosaccharomyces pombe.
  JOURNAL   Yeast. 1998 Jul;14(10):943-51.
   PUBMED   9717240
REFERENCE   2  (bases 1 to 3938)
  AUTHORS   .
  TITLE     Direct Submission
  JOURNAL   Exported Jun 16, 2022 from SnapGene Server 1.1.58
            http://www.snapgene.com
FEATURES             Location/Qualifiers
     source          1..3938
                     /organism="synthetic DNA construct"
                     /mol_type="other DNA"
     primer_bind     complement(35..52)
           

In [ ]:
# Generating primers for the ase1 insert fragment. 

#todo-manu: 
# 1. find the feature containing the CDS filtering the list of features
# 2. Use the coordinates of the CDS to design primers using pydna design
# 3. Append the cut site to the primers
# 4. Do the PCR

fwd_primer_ase1 = Dseqrecord("ACCATGTCGAC") + pombe_chromosome_I[1000:1020] # Adding a SalI cut site
rvs_primer_ase1_3_start = pombe_chromosome_I[3516:3546] + Dseqrecord("GGCGCGCCAT") # Adding a AscI cut site
rvs_primer_ase1 = rvs_primer_ase1_3_start.reverse_complement()

# Printing out the primers

print(fwd_primer_ase1.seq)
print(rvs_primer_ase1.seq)

ACCATGTCGACAAGCCATATCCAACCACCAT
ATGGCGCGCCGTTTTTTTTTAGGTGTAAATAGTCTTTTAA


In [ ]:
# Checking that the primer Tm are matching

#todo-manu: 
# 1. Check the Tm of the primers using the part that aligns with the genome only (here you are calculating the Tm including the restriction sites, which
# will not anneal to the template during the PCR)

print(tm_default(fwd_primer_ase1.seq)) # Modify the primer sequence above retroactively, if Tm not matching.
print(tm_default(rvs_primer_ase1.seq))

71.72846660888155
69.94587883755406


In [ ]:
# Performing a PCR to check that the primers are specific. An error message is returned if otherwise.

pcr_product = pcr(fwd_primer_ase1, rvs_primer_ase1, pombe_chromosome_I)

# Printing out the PCR results

print(pcr_product.format("gb"))

LOCUS       2567bp_PCR_prod         2567 bp    DNA     linear   UNK 01-JAN-1980
DEFINITION  pcr_product_description_description_rc.
ACCESSION   2567bp
VERSION     2567bp
DBLINK      BioProject: PRJNA13836
            BioSample: SAMEA3138176
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     gene            2..2539
                     /gene="ase1"
                     /locus_tag="SPOM_SPAPB1A10.09"
     5'UTR           2..174
                     /gene="ase1"
                     /locus_tag="SPOM_SPAPB1A10.09"
     CDS             join(175..598,646..2417)
                     /gene="ase1"
                     /locus_tag="SPOM_SPAPB1A10.09"
                     /codon_start=1
                     /product="antiparallel microtubule cross-linking factor
                     Ase1"
                     /protein_id="CAC21482.1"
                     /translation="MQTVMMDDIQSTDSIAEKDNHSNNESNFTWKAFREQVEKHFSKIE
                     RLHQVLGTDGDNS

In [ ]:
# Cleaving the cloning vector with restriction enzymes

plamsid_digests = vector.cut(SalI, AscI)

# Cleaving the gene fragment with restriction enzymes

gene_digests = Dseqrecord(pcr_product).cut(SalI, AscI)

# Printing out the digests
print(plamsid_digests) 
print(gene_digests)

(Dseqrecord(-30), Dseqrecord(-3916))
(Dseqrecord(-10), Dseqrecord(-2557), Dseqrecord(-8))


In [ ]:
# Ligating, then circularising the synthetic plasmid

Synthetic_Vector = plamsid_digests[1] + gene_digests [1]
Synthetic_Vector = Synthetic_Vector.looped()

# Printing out the completed cloning vector

print(Synthetic_Vector.format("gb"))

LOCUS       name                    6465 bp    DNA     circular UNK 01-JAN-1980
DEFINITION  description.
ACCESSION   id
VERSION     id
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     gene            52..1408
                     /label="kanMX"
                     /note="yeast selectable marker conferring kanamycin
                     resistance (Wach et al., 1994)"
     promoter        52..395
                     /label="TEF promoter"
                     /note="Ashbya gossypii TEF promoter"
     CDS             396..1205
                     /codon_start=1
                     /gene="aph(3')-Ia"
                     /product="aminoglycoside phosphotransferase"
                     /label="KanR"
                     /note="confers resistance to kanamycin"
                     /translation="MGKEKTHVSRPRLNSNMDADLYGYKWARDNVGQSGATIYRLYGKP
                     DAPELFLKHGKGSVANDVTDEMVRLNWLTEFMPLPTIKHFIRTPDDAWLLTTAIPGKTA
               